In [2]:
import openrouteservice

# Connect to local ORS
client = openrouteservice.Client(base_url='http://localhost:8082/ors')

# Two random points in Martinique (Longitude, Latitude format)
coords = [[-61.0242, 14.6415], [-61.0735, 14.6021]]

# Request driving directions
routes = client.directions(
    coordinates=coords, 
    profile='driving-car', 
    format='geojson'
)

print(routes)


{'type': 'FeatureCollection', 'metadata': {'attribution': 'openrouteservice.org, OpenStreetMap contributors, tmc - BASt', 'service': 'routing', 'timestamp': 1740753400779, 'query': {'coordinates': [[-61.0242, 14.6415], [-61.0735, 14.6021]], 'profile': 'driving-car', 'profileName': 'driving-car', 'format': 'geojson'}, 'engine': {'version': '9.1.0', 'build_date': '2025-01-29T12:41:18Z', 'graph_date': '2025-02-27T17:45:25Z'}}, 'features': [{'bbox': [-61.07457, 14.602148, -61.01643, 14.641478], 'type': 'Feature', 'properties': {'segments': [{'distance': 10466.4, 'duration': 839.3, 'steps': [{'distance': 191.7, 'duration': 46.0, 'type': 11, 'instruction': 'Head south', 'name': '-', 'way_points': [0, 11]}, {'distance': 621.5, 'duration': 49.7, 'type': 1, 'instruction': 'Turn right onto D 14', 'name': 'D 14', 'way_points': [11, 27]}, {'distance': 370.1, 'duration': 26.6, 'type': 4, 'instruction': 'Turn slight left onto D 13', 'name': 'D 13', 'way_points': [27, 45]}, {'distance': 1026.8, 'dura

In [ ]:
import io
import sys
import zipfile

import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import requests
from shapely.geometry import MultiPolygon, Point, Polygon, shape
from tqdm.notebook import tqdm

from modules import download_bpe, download_carreaus

In [4]:
import logging

# Configure logging: set level, format, and output file (if desired)
logging.basicConfig(
    level=logging.DEBUG,  # Capture all levels: DEBUG, INFO, WARNING, ERROR, CRITICAL
    format='%(asctime)s [%(levelname)s] %(message)s',  # Log format with timestamp and level
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Log messages at different severity levels
logging.debug("This is a debug message.")
logging.info("This is an info message.")
logging.warning("This is a warning message.")
logging.error("This is an error message.")
logging.critical("This is a critical message.")

# Example function with logging
def divide(a, b):
    logging.info(f"Dividing {a} by {b}")
    try:
        result = a / b
        logging.debug(f"Result: {result}")
        return result
    except ZeroDivisionError:
        logging.error("Attempted to divide by zero!")
        return None

# Run the example function
divide(10, 2)
divide(10, 0)


2025-02-28 15:36:42 - INFO - This is an info message.
2025-02-28 15:36:42 - WARNING - This is a warning message.
2025-02-28 15:36:42 - ERROR - This is an error message.
2025-02-28 15:36:42 - CRITICAL - This is a critical message.
2025-02-28 15:36:42 - INFO - Dividing 10 by 2
2025-02-28 15:36:42 - INFO - Dividing 10 by 0
2025-02-28 15:36:42 - ERROR - Attempted to divide by zero!


In [5]:

# Définir l'URL de base de votre instance locale d'ORS
ORS_URL = "http://localhost:8082/ors/isochrones/"

def split_by_dom(df_bpe):
    df_filtered = df_bpe.loc[
        df_bpe["DOM"].isin(["D", "F", "C"]) |
        df_bpe["TYPEQU"].isin(["E108", "E109"]) |
        df_bpe["SDOM"].isin(["A1", "B1", "B2"])
    ].copy()
    df_filtered["ID"] = range(1, len(df_filtered) + 1)
    cols_to_keep = [
        "ID", "AN", "NOMRS", "CNOMRS", "NUMVOIE", "INDREP", "TYPVOIE", "LIBVOIE",
        "CADR", "LIBCOM", "CODPOS", "DEPCOM", "REG", "DOM", "SDOM", "TYPEQU", "SIRET", 
        "LAMBERT_X", "LAMBERT_Y", "LONGITUDE", "LATITUDE"
    ]
    existing_cols = [col for col in cols_to_keep if col in df_filtered.columns]
    df_filtered = df_filtered[existing_cols].copy()

    def process_region(df, region_code):
        df_region = df[df["DEPCOM"].astype(str).str.startswith(region_code)].copy()
        df_d_dom = df_region[df_region["SDOM"].isin(["D1", "D3", "D4", "D6", "D7"])].copy()

        def combine_typequ(series):
            unique_vals = series.dropna().unique()
            return ", ".join(map(str, unique_vals))

        if not df_d_dom.empty:
            df_d_dom["TYPEQU"] = df_d_dom.groupby(["NOMRS", "LIBCOM", "DOM", "LIBVOIE"])["TYPEQU"].transform(combine_typequ)
            df_unique_D_dom = df_d_dom.drop_duplicates(subset=["NOMRS", "LIBCOM", "DOM", "LIBVOIE"])
        else:
            df_unique_D_dom = df_d_dom

        df_dom_rest = df_region[~df_region["SDOM"].isin(["D1", "D3", "D4", "D6", "D7"])]
        df_unique = pd.concat([df_dom_rest, df_unique_D_dom], ignore_index=True)
        return df_unique

    martinique_df = process_region(df_filtered, "972")
    reunion_df = process_region(df_filtered, "974")
    
    return martinique_df, reunion_df



def map_carreaus_osrm(carr_geo, df):
    logging.info("Starting map_carreaus_osrm function...")

    headers = {"Content-Type": "application/json"}
    transport_methods = ["driving-car", "cycling-regular", "foot-walking"]

    # Initialize transport mode scores
    for mode in transport_methods:
        carr_geo[f"{mode}_score"] = 0

    logging.info("Transport mode scores initialized.")

    temp = carr_geo[0:100]

    # Loop through each geographic entity
    for idx, row in tqdm(temp[0:100].iterrows(), total=len(temp)):
        lat, lon = row['latitude'], row['longitude']
        logging.info(f"Processing entity {idx} at coordinates ({lat}, {lon})")

        payload = {
            "locations": [[lon, lat]],
            "range": [900],
            "range_type": "time"
        }
    
        gdfs = []
        
        for mode in transport_methods:
            travel_url = f"{ORS_URL}{mode}"
            logging.info(f"Requesting isochrone for {mode} mode...")

            try:
                osrm_response = requests.post(travel_url, json=payload, headers=headers)
                
                if osrm_response.status_code != 200:
                    logging.error(f"Error for {mode} at {idx}: {osrm_response.text}")
                    carr_geo.at[idx, f"{mode}_score"] = None
                    continue
                
                isochrone_geojson = osrm_response.json()["features"][0]["geometry"]
                isochrone_polygon = shape(isochrone_geojson)

                logging.info(f"Isochrone received for {mode}, processing...")

                # Clip isochrone with Martinique boundary
                try:
                    martinique_boundary = gpd.read_file(
                        r"D:\ECOLAB\GD4H\Filosofi2017_carreaux_200m_shp\Filosofi2017_carreaux_200m_shp\Filosofi2017_carreaux_200m_mart.shp"
                    ).to_crs(epsg=5794)

                    isochrone_polygon = isochrone_polygon.intersection(martinique_boundary.unary_union)
                    logging.info(f"Isochrone successfully clipped for {mode}.")
                except Exception as e:
                    logging.warning(f"Failed to clip isochrone for {mode} at {idx}: {e}")

                isochrone_gdf = gpd.GeoDataFrame(
                    {"transport_mode": [mode], "geometry": [isochrone_polygon]},
                    crs="EPSG:5794"
                )
                gdfs.append(isochrone_gdf)
            
                # Compute score: count BPE points inside the isochrone
                carr_geo.at[idx, f"{mode}_score"] = df.geometry.within(isochrone_polygon).sum()
                logging.info(f"Score computed for {mode}: {carr_geo.at[idx, f'{mode}_score']}")

            except Exception as e:
                logging.error(f"Unexpected error while processing {mode} at {idx}: {e}")


            
    #     if gdfs:
    #         merged_isochrones_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
    #     else:
    #         merged_isochrones_gdf = None            
        
    #     # Affichage de l'isochrone et du centroïde
    #     fig, ax = plt.subplots(figsize=(8, 8))
    #     if merged_isochrones_gdf is not None:
    #         merged_isochrones_gdf.plot(ax=ax, edgecolor="red", facecolor="none", linestyle="--", label="Isochrone")
    #     ax.scatter(lon, lat, color="blue", marker="o", label="Centroid (Origine)")
    #     plt.legend()
    #     plt.show()
        
    # # Affichage des cartes choroplèth pour chaque mode
    # for mode in transport_methods:
    #     fig, ax = plt.subplots(figsize=(10, 10))
    #     carr_geo.plot(column=f"{mode}_score", ax=ax, cmap="OrRd", edgecolor="black", legend=True, alpha=0.5)
    #     plt.title(f"Carte Choroplèth pour {mode}")
    #     plt.show()
    logging.info("Finished processing all entities.")
    return carr_geo

logging.info('Starting')

logging.info('Creating map')
# Création d'une carte Folium centrée sur la Martinique
m = folium.Map(location=[14.6415, -61.0242], zoom_start=10)

logging.info('Map Done')

# Chargement et traitement des données
logging.info('Downloading Stuff')
# df_bpe = download_bpe()
df_bpe = pd.read_csv(r'data\insee\bpe_unique_Martinique_Reunion.csv')
carreaus = download_carreaus(epsg=5794)
logging.info('Download Done')

carreaus = map_carreaus_osrm(carreaus, df_bpe)
gdf = gpd.GeoDataFrame(df_bpe, geometry="geometry", crs="EPSG:5794")
pd.set_option("display.max_columns", None)

# Sauvegarde de la carte isochrone au format HTML
m.save("isochrone_map.html")


2025-02-28 15:36:42 - INFO - Starting
2025-02-28 15:36:42 - INFO - Creating map
2025-02-28 15:36:42 - INFO - Map Done
2025-02-28 15:36:42 - INFO - Downloading Stuff
2025-02-28 15:36:43 - INFO - Download Done
2025-02-28 15:36:43 - INFO - Starting map_carreaus_osrm function...
2025-02-28 15:36:43 - INFO - Transport mode scores initialized.


  0%|          | 0/100 [00:00<?, ?it/s]

2025-02-28 15:36:43 - INFO - Processing entity 0 at coordinates (-60.87710643657253, 14.397108641350352)
2025-02-28 15:36:43 - INFO - Requesting isochrone for driving-car mode...
2025-02-28 15:36:43 - ERROR - Error for driving-car at 0: {"type":"about:blank","title":"Not Found","status":404,"detail":"No static resource isochrones/driving-car.","instance":"/ors/isochrones/driving-car","properties":null}
2025-02-28 15:36:43 - INFO - Requesting isochrone for cycling-regular mode...
2025-02-28 15:36:43 - ERROR - Error for cycling-regular at 0: {"type":"about:blank","title":"Not Found","status":404,"detail":"No static resource isochrones/cycling-regular.","instance":"/ors/isochrones/cycling-regular","properties":null}
2025-02-28 15:36:43 - INFO - Requesting isochrone for foot-walking mode...
2025-02-28 15:36:43 - ERROR - Error for foot-walking at 0: {"type":"about:blank","title":"Not Found","status":404,"detail":"No static resource isochrones/foot-walking.","instance":"/ors/isochrones/foot-

ValueError: Unknown column geometry

In [ ]:
import json

import openrouteservice
import pandas as pd
from tqdm.notebook import tqdm

# # --- User-specified parameters ---
# # You can change these to your preferred values.
# profile = input("Enter profile (e.g., driving-car, cycling-regular, foot-walking): ").strip()
# geography = input("Enter geography (for naming, e.g., Martinique): ").strip()
# range_val = input("Enter range value (e.g., 300): ").strip()
# interval_val = input("Enter interval value (e.g., 300): ").strip()

profile = "driving-car"
geography = "Martinique"
range_val = 300
interval_val = 300

# Convert range and interval to integer values
try:
    range_val = int(range_val)
    interval_val = int(interval_val)
except Exception as e:
    print("Error: Range and interval must be numeric.")
    exit()

# --- Initialize the ORS client for your local instance ---
client = openrouteservice.Client(base_url='http://localhost:8082/ors')

# --- Build an output file name that reflects the parameters ---
output_file = f"out/results_{profile}_{geography}_{range_val}_{interval_val}.jsonl"
print(f"Results will be saved in: {output_file}")

# --- Process each row of the 'carreaus' DataFrame ---
with open(output_file, "w") as f_out:
    for idx, row in tqdm(carreaus.iterrows(), total=len(carreaus), desc="Processing points"):
        # Retrieve coordinates from DataFrame.
        # (Make sure your DataFrame has 'longitude' and 'latitude' columns.)
        lat, lon = row["longitude"], row["latitude"]
        try:
            # Request an isochrone using the user-specified parameters.
            response = client.isochrones(
                locations=[[round(lon, 6), round(lat, 6)]],
                profile=profile,
                range=[range_val],
                interval=interval_val
            )

            response['Idcar_200m'] = row['Idcar_200m']
            # Write the JSON response as a single line.
            f_out.write(json.dumps(response) + "\n")
        except Exception as e:
            print(f"Error for row {idx} with coordinates ({lon}, {lat}): {e}")



Results will be saved in: out/results_driving-car_Martinique_300_300.jsonl


Processing points:   0%|          | 0/11132 [00:00<?, ?it/s]

Error for row 5450 with coordinates (-60.848935220744366, 14.62636831066687): 500 ({'error': {'code': 3099, 'message': 'Unable to build an isochrone map.'}, 'info': {'engine': {'build_date': '2025-01-29T12:41:18Z', 'graph_version': '1', 'version': '9.1.0'}, 'timestamp': 1740756781355}})


In [100]:
import json

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import shape

data = []
with open(r"out\results_driving-car_Martinique_300_300.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))

df = pd.DataFrame(data)

features = pd.json_normalize(df['features']).rename(columns={0:'features'})

features_normalize = pd.json_normalize(features['features'])



In [117]:
# Clean up column names (if needed)
features_normalize.columns = features_normalize.columns.str.strip()

# Convert each row into a Shapely geometry.
# If the "geometry.type" column is missing, you can default to a type (e.g., "Polygon")
features_normalize['geometry'] = features_normalize.apply(
    lambda row: shape({
        'type': row.get('geometry.type', 'Polygon'),  # Use 'Polygon' as default if missing
        'coordinates': row['geometry.coordinates']
    }),
    axis=1
)

# Create a GeoDataFrame from your DataFrame using the new 'geometry' column.
isochrone_gdf = gpd.GeoDataFrame(features_normalize, crs="EPSG:5794", geometry='geometry')

isochrone_gdf['Idcar_200m'] = df['Idcar_200m']

# Assuming your DataFrame 'df_bpe' has columns 'longitude' and 'latitude'
isochrone_gdf = gpd.GeoDataFrame(
    df_bpe,
    geometry=gpd.points_from_xy(df_bpe.LONGITUDE, df_bpe.LATITUDE),
    crs="EPSG:5794"  # You can change the CRS if needed
)


In [135]:
# Assuming your DataFrame 'df_bpe' has columns 'longitude' and 'latitude'
gdf_bpe = gpd.GeoDataFrame(
    df_bpe,
    geometry=gpd.points_from_xy(df_bpe.LONGITUDE, df_bpe.LATITUDE),
    crs="EPSG:5794"  # You can change the CRS if needed
)


In [136]:
joined = gdf_bpe.sjoin(gdf, how="inner")


In [138]:
joined

,ID,AN,NOMRS,CNOMRS,NUMVOIE,INDREP,TYPVOIE,LIBVOIE,CADR,LIBCOM,...,LATITUDE,geometry,index_right,type,properties.group_index,properties.value,properties.center,geometry.coordinates,geometry.type,Idcar_200m
0,7381,2023,AJOUPA BOUILLON,NaN,NaN,NaN,BRG,BOURG,NaN,L'AJOUPA-BOUILLON,...,14.827016,POINT (-61.11217 14.82702),10637,Feature,0,300.0,"[-61.12394264520001, 14.81850116434261]","[[[-61.130565, 14.812422], [-61.130499, 14.812...",Polygon,CRS5490RES200mN1639000E701800
0,7381,2023,AJOUPA BOUILLON,NaN,NaN,NaN,BRG,BOURG,NaN,L'AJOUPA-BOUILLON,...,14.827016,POINT (-61.11217 14.82702),10638,Feature,0,300.0,"[-61.1223394466911, 14.819651872769711]","[[[-61.129774, 14.81542], [-61.129261, 14.8108...",Polygon,CRS5490RES200mN1639000E702000
0,7381,2023,AJOUPA BOUILLON,NaN,NaN,NaN,BRG,BOURG,NaN,L'AJOUPA-BOUILLON,...,14.827016,POINT (-61.11217 14.82702),10722,Feature,0,300.0,"[-61.1249498, 14.8214831]","[[[-61.129081, 14.815715], [-61.129023, 14.815...",Polygon,CRS5490RES200mN1639400E701600
0,7381,2023,AJOUPA BOUILLON,NaN,NaN,NaN,BRG,BOURG,NaN,L'AJOUPA-BOUILLON,...,14.827016,POINT (-61.11217 14.82702),10680,Feature,0,300.0,"[-61.12182811040763, 14.819794058551514]","[[[-61.129774, 14.81542], [-61.129194, 14.8109...",Polygon,CRS5490RES200mN1639200E702000
0,7381,2023,AJOUPA BOUILLON,NaN,NaN,NaN,BRG,BOURG,NaN,L'AJOUPA-BOUILLON,...,14.827016,POINT (-61.11217 14.82702),10678,Feature,0,300.0,"[-61.12558688955323, 14.820566301208377]","[[[-61.12947, 14.815652], [-61.129412, 14.8152...",Polygon,CRS5490RES200mN1639200E701600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19224,13726,2023,ATELIER PROTEGE BELLE FONTAINE,NaN,NaN,NaN,NaN,FONDS BOUCHER,NaN,BELLEFONTAINE,...,14.657788,POINT (-61.14999 14.65779),7096,Feature,0,300.0,"[-61.15490389702508, 14.665178629063245]","[[[-61.163938, 14.670535], [-61.163842, 14.670...",Polygon,CRS5490RES200mN1622000E698600
19224,13726,2023,ATELIER PROTEGE BELLE FONTAINE,NaN,NaN,NaN,NaN,FONDS BOUCHER,NaN,BELLEFONTAINE,...,14.657788,POINT (-61.14999 14.65779),7019,Feature,0,300.0,"[-61.153069408738084, 14.663302204985404]","[[[-61.160714, 14.666301], [-61.156645, 14.663...",Polygon,CRS5490RES200mN1621800E698800
19224,13726,2023,ATELIER PROTEGE BELLE FONTAINE,NaN,NaN,NaN,NaN,FONDS BOUCHER,NaN,BELLEFONTAINE,...,14.657788,POINT (-61.14999 14.65779),7155,Feature,0,300.0,"[-61.15293801396306, 14.666997819187122]","[[[-61.160262, 14.666604], [-61.156387, 14.663...",Polygon,CRS5490RES200mN1622200E698800
19224,13726,2023,ATELIER PROTEGE BELLE FONTAINE,NaN,NaN,NaN,NaN,FONDS BOUCHER,NaN,BELLEFONTAINE,...,14.657788,POINT (-61.14999 14.65779),6858,Feature,0,300.0,"[-61.153548890190834, 14.65960376866186]","[[[-61.167009, 14.674907], [-61.166635, 14.674...",Polygon,CRS5490RES200mN1621400E698800


In [140]:
joined.groupby('Idcar_200m').agg(
    {'ID':'count'}
).sort_values(by='ID')

,ID
Idcar_200m,
CRS5490RES200mN1638600E703600,1
CRS5490RES200mN1605200E734600,1
CRS5490RES200mN1605200E734400,1
CRS5490RES200mN1605200E734200,1
CRS5490RES200mN1621800E727800,1
...,...
CRS5490RES200mN1616000E707600,979
CRS5490RES200mN1616600E707600,1002
CRS5490RES200mN1615800E707800,1010


In [141]:
carreaus.loc[carreaus['Idcar_200m']=='CRS5490RES200mN1615600E708200']

,Idcar_200m,I_est_200,Idcar_1km,I_est_1km,Idcar_nat,Groupe,Ind,Men_1ind,Men_5ind,Men_prop,...,Ind_inc,Men_pauv,Men,lcog_geo,geometry,longitude,latitude,driving-car_score,cycling-regular_score,foot-walking_score
4494,CRS5490RES200mN1615600E708200,0,CRS5490RES1000mN1615000E708000,0,CRS5490RES1000mN1615000E708000,1018338,93.0,32.0,2.0,7.0,...,0.0,26.0,53.0,97209,"POLYGON ((14.60569 -61.06721, 14.6075 -61.0671...",14.606585,-61.066274,0.0,0.0,0.0
